In [ ]:
import os
import time
import requests
import fitz  # PyMuPDF
import pandas as pd
from io import BytesIO
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ▶ 설정
CHROMEDRIVER_PATH = r"C:\Aicamp\chromedriver-win64\chromedriver-win64\chromedriver.exe"
BASE_URL = "https://opengov.seoul.go.kr"
SAVE_PATH = "press_crawled.csv"

# ▶ 셀레니움 드라이버 설정
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0")
# options.add_argument("--headless")  # 필요 시 사용

driver = webdriver.Chrome(service=Service(CHROMEDRIVER_PATH), options=options)
wait = WebDriverWait(driver, 10)

data = []
article_id = 1

for i in range(1, 3):
    page_url = f"{BASE_URL}/press/list?page={i}"
    print(f"✅ 리스트 접속 성공: {i}페이지")
    driver.get(page_url)
    time.sleep(1)

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table tbody tr td.data-title.aLeft > a")))
        elements = driver.find_elements(By.CSS_SELECTOR, "table tbody tr td.data-title.aLeft > a")
        detail_links = [el.get_attribute("href") for el in elements]
        print(f"📌 상세페이지 링크 수: {len(detail_links)}개")

        for link in detail_links:
            try:
                driver.get(link)
                time.sleep(1.5)
                soup = BeautifulSoup(driver.page_source, "html.parser")

                # 제목 수집
                title_tag = soup.select_one("#content > div > div.view-content.view-content-article > h3")
                if not title_tag:
                    print("❌ 제목 없음, 스킵")
                    continue
                title = title_tag.get_text(strip=True)

                # PDF 링크 수집
                pdf_tag = soup.select_one("#content > div > div.view-content.view-content-article a[href$='.pdf']")
                if pdf_tag:
                    href = pdf_tag.get("href")
                    if isinstance(href, str):
                        pdf_url = BASE_URL + href
                        print(f"🔎 상세페이지: {link}")
                        print(f"👉 제목: {title}")
                        print(f"👉 PDF 링크: {pdf_url}")
                    else:
                        print(f"❌ PDF href 오류: {title} / {link}")
                        continue
                else:
                    pdf_tag = soup.select_one("#content > div > div.view-content.view-content-article > div:nth-child(8) > ul > li:nth-child(1) > span > a:nth-child(2) > i")
                    if pdf_tag:
                        href = pdf_tag.get("href")
                        if isinstance(href, str):
                            pdf_url = BASE_URL + href
                            print(f"🔎 상세페이지: {link}")
                            print(f"👉 제목: {title}")
                            print(f"👉 PDF 링크: {pdf_url}")
                        else:
                            print(f"❌ PDF href 오류: {title} / {link}")
                            continue
                    else:
                        print(f"❌ PDF 없음: {title} / {link}")
                        continue

                # PDF 요청 및 파싱
                res = requests.get(pdf_url)
                doc = fitz.open(stream=BytesIO(res.content), filetype="pdf")
                text = "\n".join([page.get_text() for page in doc])

                data.append({
                    "id": article_id,
                    "질문": title,
                    "본문": text.strip()
                })
                article_id += 1

            except Exception as e:
                print(f"❌ 상세페이지 처리 오류: {e}")
                continue

    except Exception as e:
        print(f"❌ 전체 크롤링 오류 (page {i}): {e}")
        continue

driver.quit()

# ▶ CSV 저장
df = pd.DataFrame(data)
df.to_csv(SAVE_PATH, index=False, encoding="utf-8-sig")
print(f"\n✅ 크롤링 완료: 총 {len(df)}건 수집 → {SAVE_PATH}")

In [7]:
import pandas as pd

df = pd.read_csv("press_crawled.csv")
df


,id,질문,본문
0,1,"서울시, 빅데이터로 도시정책 예측…생활밀착형 맞춤 정책 추진",- 1 -\n2025. 7. 18.(금) 조간용\n이 보도자료는 2025년 7월 1...
1,2,(자료제공) 잠실종합운동장 방문객 대중교통 이용 당부,- 1 -\n자료제공 : 2025. 7. 17.(목)\n이 보도 자료는 배포 즉시 ...
2,3,"서울시, 소비쿠폰 지급대상 외국인 주민의 원활한 신청 위한 지원체계 구축",- 1 -\n2025. 7. 18.(금) 조간용\n이 보도자료는 2025년 7월 1...
3,4,"(자료제공) “서울마음편의점, 진정한 인간적 연결 상징 공간”… 英 가디언 집중 조명",- 1 -\n자료제공 : 2025. 7. 17.(목)\n이 보도자료는 배포 즉시 보...
4,5,"서울시, 차세대 소방안전지도시스템 본격 가동, 클라우드 기술로 재난현장 대응력 강화 나서",- 1 -\n□서울시소방재난본부(본부장 권혁민)는17일(목) 오후본부청사에서공공\n...
5,6,(자료제공) 서울시장 사후동정 - 17일(목) 아부다비 행정교통국 의장 면담,- 1 -\n 서울특별시장 동정자료 \n■ 매수 : 5매 ■ 사진 : 5매 ...
6,7,"‘규제철폐 23호’… 서울시, 자치구 건축심의 대상 60％ 대폭 축소 건설활력 높인다",- 1 -\n2025. 7. 17.(목) 조간용\n이 보도 자료는 2025년 7월 ...
7,8,(석간) 제3차 도시건축공동위원회 수권분과위원회 개최결과,- 1 -\n2025. 7. 16.(수) 석간용\n이 보도자료는 2025년 7월 1...
8,9,365일 도심 속 ‘숲캉스’… 서울 첫 자연휴양림 수락산 ‘수락 휴(休)’ 개장,- 1 -\n2025. 7. 17.(목) 조간용\n이 보도자료는 2025년 7월 1...
9,10,신길동 314-14 신통기획 확정…10년 정체 개발 재시동,- 1 -\n2025. 7. 17.(목) 조간용\n이 보도자료는 2025년 7월 1...
